In [1]:
%load_ext autoreload
%autoreload 2
from pesco.eval import load_evaluator_from_file

evaluator = load_evaluator_from_file("../data/cpachecker_labels.jsonl")

In [2]:
from pesco.utils import load_dataset

CST_FEATURES = "../data/datasets/svcomp22_cst_tool.jsonl"
LABELS = "../data/cpachecker_labels.jsonl"

cst_dataset = load_dataset(CST_FEATURES, LABELS, fill_unknown = True)

In [3]:
# Identify trivial instances
nontrivial = cst_dataset.labels.max(axis = 1) !=  0
nontrivial = set(instance for i, instance in enumerate(cst_dataset.instance_index) if nontrivial[i])
print(len(nontrivial))

4602


In [4]:
common_instances = list(sorted(nontrivial))

from pesco.utils import Dataset, _index_elements

cst_index = {k: i for i, k in enumerate(cst_dataset.instance_index)}
cst_indices = [cst_index[k] for k in common_instances]
cst_dataset  = Dataset(*map(lambda x: _index_elements(x, cst_indices), cst_dataset[:-1]), cst_dataset.label_index)

print(cst_dataset.embedding.shape)

(4602, 128)


In [5]:
from pesco.utils import stratified_split

train_dataset, test_dataset = stratified_split(cst_dataset, 0.3)
train_dataset.embedding.shape, test_dataset.embedding.shape

((3254, 128), (1348, 128))

In [6]:
from sklearn.decomposition import  PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

n_components = 128
lsa = make_pipeline(StandardScaler(), PCA(n_components = n_components, whiten = True))
train_embedding = lsa.fit_transform(train_dataset.embedding)
test_embedding = lsa.transform(test_dataset.embedding)

# Baselines #1

In [7]:
from tqdm import trange
from sklearn.linear_model import LogisticRegression

models = []
for i in trange(train_dataset.labels.shape[1]):
    labels = train_dataset.labels[:, i] 
    clf    =  LogisticRegression(C = 0.1)
    clf.fit(train_embedding, labels)
    models.append(clf)



100%|██████████| 6/6 [00:00<00:00, 28.35it/s]


In [8]:
import numpy as np

predictions = []
for i in trange(test_dataset.labels.shape[1]):
    prediction = models[i].predict_proba(test_embedding)[:, 1]
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

100%|██████████| 6/6 [00:00<00:00, 288.31it/s]


In [9]:
# Evaluate
indices = predictions.argmax(axis = 1)
tools   = [test_dataset.label_index[i] for i in indices]
mapping = {test_dataset.instance_index[i]: tool for i, tool in enumerate(tools)}

b1_result = evaluator.eval(mapping)

print("Solve: ", b1_result.score() / test_dataset.embedding.shape[0])
print(b1_result)


Solve:  0.93026706231454
Result( 1254 / 1348 )


# Baseline #2: Predict PAR10

In [10]:
runtimes = train_dataset.runtimes
mask     = train_dataset.labels

weight = (runtimes * mask) + ((1 - mask) * 9000)
log_weight = np.log10(weight)

In [11]:
from tqdm import trange
from sklearn.linear_model import RidgeCV

models = []
for i in trange(train_dataset.labels.shape[1]):
    labels = train_dataset.labels[:, i]
    sample_weight = weight[:, i]
    clf    =  RidgeCV(alphas = [1e-3, 1e-2, 1e-1, 1])
    clf.fit(train_embedding, sample_weight)
    models.append(clf)

100%|██████████| 6/6 [00:01<00:00,  3.37it/s]


In [12]:
import numpy as np

predictions = []
for i in trange(test_dataset.labels.shape[1]):
    prediction = models[i].predict(test_embedding)
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

100%|██████████| 6/6 [00:00<00:00, 181.35it/s]


In [13]:
# Evaluate
indices = predictions.argmin(axis = 1)
solve_mask = np.zeros(predictions.shape)
solve_mask[np.arange(indices.shape[0]), indices] = 1

solvabilty = (solve_mask * test_dataset.labels).max(axis = 1).mean()

tools   = [test_dataset.label_index[i] for i in indices]
mapping = {test_dataset.instance_index[i]: tool for i, tool in enumerate(tools)}

b2_result = evaluator.eval(mapping)

print("Solve (sim): ", b2_result.score() / test_dataset.embedding.shape[0])
print(b2_result)


Solve (sim):  0.9317507418397626
Result( 1256 / 1348 )


# Baseline #3: Ranking

In [14]:
runtimes = train_dataset.runtimes
mask     = train_dataset.labels
weight = (runtimes * mask) + ((1 - mask) * 9000)

rank_pos    = []
rank_weight = []

for i in range(weight.shape[1] - 1):
    for j in range(i + 1, weight.shape[1]):
        rank_pos.append((i, j))
        rank_weight.append((weight[:, j] - weight[:, i]))

rank_weight = np.stack(rank_weight).transpose()
rank_labels, rank_sample_weight = np.clip(np.sign(rank_weight), 0, 1), np.abs(rank_weight)

In [15]:
from tqdm import trange
from sklearn.linear_model import LogisticRegression

models = []
for i in trange(rank_labels.shape[1]):
    labels = rank_labels[:, i] 
    sample_weight = rank_sample_weight[:, i]

    if labels.min() != labels.max():
        clf    =  LogisticRegression(C = 0.1, max_iter = 10_000)
        clf.fit(train_embedding, labels, sample_weight = sample_weight)
    else:
        clf = labels.min()

    models.append(clf)

100%|██████████| 15/15 [00:06<00:00,  2.16it/s]


In [16]:
import numpy as np

predictions = []
for i in trange(len(models)):
    model = models[i]
    try:
        prediction = model.predict_proba(test_embedding)[:, 1]
    except Exception:
        prediction = np.array([model] * test_embedding.shape[0])
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()
predictions.shape

100%|██████████| 15/15 [00:00<00:00, 393.67it/s]


(1348, 15)

In [17]:
scores = np.zeros((test_dataset.labels.shape))
for i, (p, k) in enumerate(rank_pos):
    pred = predictions[:, i]

    scores[:, p] += pred
    scores[:, k] += (1 - pred)

In [18]:
# Evaluate
indices = scores.argmax(axis = 1)
tools   = [test_dataset.label_index[i] for i in indices]
mapping = {test_dataset.instance_index[i]: tool for i, tool in enumerate(tools)}

b3_result = evaluator.eval(mapping)

print("Solve (b1): ", b1_result.score() / test_dataset.embedding.shape[0])
print("Solve (b2): ", b2_result.score() / test_dataset.embedding.shape[0])
print("Solve (b3): ", b3_result.score() / test_dataset.embedding.shape[0])
print(b3_result)

Solve (b1):  0.93026706231454
Solve (b2):  0.9317507418397626
Solve (b3):  0.9287833827893175
Result( 1252 / 1348 )


# Portfolio

In [19]:
runtimes = train_dataset.runtimes
mask     = train_dataset.labels
weight = (runtimes * mask) + ((1 - mask) * 9000)

vbs = weight.min(axis = 1)

In [20]:
tools    = []
models   = []
rank_pos = []
regret = np.full((train_dataset.embedding.shape[0],), 9000)  - vbs
print("Current regret:", regret.mean())

Current regret: 8947.658623054735


In [21]:
# Compute first model

advantages = []
for i in range(weight.shape[1]):
    performance = weight[:, i]
    cregret     = performance - vbs
    advantage   = regret - cregret
    advantages.append(advantage)

candidate, advantage = max(enumerate(advantages), key = lambda x: x[1].mean())
tools.append(candidate)
regret = regret - advantage
print("Tools:", [train_dataset.label_index[i] for i in tools])
print("Current regret:", regret.mean())

Tools: ['ki']
Current regret: 1974.9199483111734


In [22]:
def max_corr(index):
    compare_weights = np.stack([weight[:, index]] + [weight[:, i] for i in tools])
    covariance = np.corrcoef(compare_weights)
    index_cov  = covariance[0, 1:]
    return index_cov.max()
    

advantages = []
for i in range(weight.shape[1]):
    if max_corr(i) >= 0.9:
        advantages.append(np.array([0] * weight.shape[0]))
        continue # This is not always possible (better option?)
    performance = weight[:, i]
    cregret     = performance - vbs
    advantage   = regret - cregret
    advantage   = np.clip(advantage, 0, None)
    advantages.append(advantage)

candidate, advantage = max(enumerate(advantages), key = lambda x: x[1].mean())
print(train_dataset.label_index[candidate], advantage.mean())

vaitp 1532.1492602958506


In [23]:
# Create classifier

from tqdm import tqdm

for p in tqdm(tools):
    rank_pos.append((p, candidate))
    advantage = weight[:, candidate] - weight[:, p]
    labels, sample_weight = np.clip(np.sign(advantage), 0, 1), np.abs(advantage)
    try:
        clf = LogisticRegression(C = 0.1, max_iter = 10_000)
        clf.fit(train_embedding, labels, sample_weight = sample_weight)
    except Exception:
        clf = labels.min()

    models.append(clf)

tools.append(candidate)

100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


In [24]:
# Compute new predictions
predictions = []
for i in trange(len(models)):
    model = models[i]
    try:
        prediction = model.predict_proba(train_embedding)[:, 1]
    except Exception:
        prediction = np.array([model] * train_embedding.shape[0])
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

scores = np.zeros((weight.shape))
for i, (p, k) in enumerate(rank_pos):
    pred = predictions[:, i]

    scores[:, p] += pred
    scores[:, k] += (1 - pred)

selection = scores.argmax(axis = 1)

100%|██████████| 1/1 [00:00<00:00, 115.54it/s]


In [25]:
selected_weight = weight[np.arange(selection.shape[0]), selection]
regret = selected_weight - vbs
print("Tools:", [train_dataset.label_index[i] for i in tools])
print("Current regret:", np.mean(regret))

Tools: ['ki', 'vaitp']
Current regret: 642.6346333086556


In [26]:
# Compute new predictions
predictions = []
for i in trange(len(models)):
    model = models[i]
    try:
        prediction = model.predict_proba(test_embedding)[:, 1]
    except Exception:
        prediction = np.array([model] * test_embedding.shape[0])
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

scores = np.zeros((test_dataset.labels.shape))
for i, (p, k) in enumerate(rank_pos):
    pred = predictions[:, i]

    scores[:, p] += pred
    scores[:, k] += (1 - pred)

selection = scores.argmax(axis = 1)

# Evaluate
indices = scores.argmax(axis = 1)
solve_mask = np.zeros(scores.shape)
solve_mask[np.arange(indices.shape[0]), indices] = 1

tool_names   = [test_dataset.label_index[i] for i in indices]
mapping = {test_dataset.instance_index[i]: tool for i, tool in enumerate(tool_names)}

b4_result = evaluator.eval(mapping)

print("Solve (b1): ", b1_result.score() / test_dataset.embedding.shape[0])
print("Solve (b2): ", b2_result.score() / test_dataset.embedding.shape[0])
print("Solve (b3): ", b3_result.score() / test_dataset.embedding.shape[0])
print("Solve (b4): ", b4_result.score() / test_dataset.embedding.shape[0])
print(b4_result)

100%|██████████| 1/1 [00:00<00:00, 274.80it/s]

Solve (b1):  0.93026706231454
Solve (b2):  0.9317507418397626
Solve (b3):  0.9287833827893175
Solve (b4):  0.8887240356083086
Result( 1198 / 1348 )


# Baseline #5: Sequential compositions

In [27]:
from pesco.optim import optimize_portfolio

seqp = optimize_portfolio(train_dataset.labels, train_dataset.runtimes)
seqp = [(train_dataset.label_index[solver], runtime) for solver, runtime in seqp.portfolio]
print(seqp)

[('pa', 60), ('bmc', 180), ('vaitp', 480), ('ki', 900)]


In [28]:
mapping = {instance: seqp for instance in test_dataset.instance_index}
b5_result = evaluator.eval(mapping)

print("Solve (b1): ", b1_result.score() / test_dataset.embedding.shape[0])
print("Solve (b2): ", b2_result.score() / test_dataset.embedding.shape[0])
print("Solve (b3): ", b3_result.score() / test_dataset.embedding.shape[0])
print("Solve (b4): ", b4_result.score() / test_dataset.embedding.shape[0])
print("Solve (b5): ", b5_result.score() / test_dataset.embedding.shape[0])
print(b5_result)

Solve (b1):  0.93026706231454
Solve (b2):  0.9317507418397626
Solve (b3):  0.9287833827893175
Solve (b4):  0.8887240356083086
Solve (b5):  0.9339762611275965
Result( 1259 / 1348 )


# Baseline #6: Selection of Sequential Compositions

In [65]:
runtimes = train_dataset.runtimes
mask     = train_dataset.labels
weight = (runtimes * mask) + ((1 - mask) * 9000)

vbs = weight.min(axis = 1)

In [66]:
tools    = []
tool_weights = []

models   = []
rank_pos = []
regret = np.full((train_dataset.embedding.shape[0],), 9000)  - vbs
print("Current regret:", regret.mean())

Current regret: 8947.658623054735


In [67]:

seqp = optimize_portfolio(train_dataset.labels, train_dataset.runtimes, penalty = regret)
seqp = [(train_dataset.label_index[solver], runtime) for solver, runtime in seqp.portfolio]
tool = ",".join(":".join([str(_b) for _b in b]) for b in seqp)
tool

'pa:60,bmc:180,vaitp:480,ki:900'

In [68]:
# Eval tool
mapping = {instance: seqp for instance in train_dataset.instance_index}
result  = evaluator.eval(mapping)
tool_solve = [r[1].startswith(str(r[2]).lower()) for r in result]
tool_solve = np.array([1 if t else 0 for t in tool_solve])
tool_runtimes = np.array([r[3] for r in result])
tool_weight   = tool_runtimes * tool_solve + 9000 * (1 - tool_solve)
regret = tool_weight - vbs

tools.append(tool)
tool_weights.append(tool_weight)

print("Current regret:", regret.mean())

Current regret: 641.4163920719972


In [95]:
seqp = optimize_portfolio(train_dataset.labels, train_dataset.runtimes, penalty = regret)
seqp = [(train_dataset.label_index[solver], runtime) for solver, runtime in seqp.portfolio]
tool = ",".join(":".join([str(_b) for _b in b]) for b in seqp)
tool

'ki:900'

In [96]:
mapping = {instance: seqp for instance in train_dataset.instance_index}
result  = evaluator.eval(mapping)
tool_solve = [r[1].startswith(str(r[2]).lower()) for r in result]
tool_solve = np.array([1 if t else 0 for t in tool_solve])
tool_runtimes = np.array([r[3] for r in result])
tool_weight   = tool_runtimes * tool_solve + 9000 * (1 - tool_solve)
tool_weight

array([6.45146552e+00, 4.73117862e+00, 9.00000000e+03, ...,
       4.10204674e+01, 4.24808199e+01, 1.08716914e+01])

In [97]:
compare_weights = np.stack([tool_weight] + [weight for weight in tool_weights])
covariance = np.corrcoef(compare_weights)
index_cov  = covariance[0, 1:]
index_cov.max()

1.0

In [91]:
# Create classifier

from tqdm import tqdm

for i, p in enumerate(tqdm(tools)):
    rank_pos.append((i, len(tools)))
    advantage = tool_weight - tool_weights[i]
    labels, sample_weight = np.clip(np.sign(advantage), 0, 1), np.abs(advantage)
    try:
        clf = LogisticRegression(C = 0.1, max_iter = 10_000)
        clf.fit(train_embedding, labels, sample_weight = sample_weight)
    except Exception:
        clf = labels.min()

    models.append(clf)

tools.append(tool)
tool_weights.append(tool_weight)

100%|██████████| 4/4 [00:01<00:00,  2.20it/s]


In [92]:
# Compute new predictions
predictions = []
for i in trange(len(models)):
    model = models[i]
    try:
        prediction = model.predict_proba(train_embedding)[:, 1]
    except Exception:
        prediction = np.array([model] * train_embedding.shape[0])
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

scores = np.zeros((predictions.shape[0], len(tools)))
for i, (p, k) in enumerate(rank_pos):
    pred = predictions[:, i]

    scores[:, p] += pred
    scores[:, k] += (1 - pred)

selection = scores.argmax(axis = 1)

100%|██████████| 10/10 [00:00<00:00, 267.20it/s]


In [93]:
cweight = np.stack(tool_weights).transpose()
selected_weight = cweight[np.arange(selection.shape[0]), selection]
regret = selected_weight - vbs
print("Tools:", tools)
print("Current regret:", np.mean(regret))

Tools: ['pa:60,bmc:180,vaitp:480,ki:900', 'va:420,ki:420,symbolic:900', 'ki:900', 'vaitp:840,bmc:900', 'symbolic:120,pa:480,bmc:900']
Current regret: 88.09078617217578


In [94]:
# Compute new predictions
predictions = []
for i in trange(len(models)):
    model = models[i]
    try:
        prediction = model.predict_proba(test_embedding)[:, 1]
    except Exception:
        prediction = np.array([model] * test_embedding.shape[0])
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

scores = np.zeros((predictions.shape[0], len(tools)))
for i, (p, k) in enumerate(rank_pos):
    pred = predictions[:, i]

    scores[:, p] += pred
    scores[:, k] += (1 - pred)

selection = scores.argmax(axis = 1)

# Evaluate
tool_names   = [tools[i] for i in selection]
mapping = {test_dataset.instance_index[i]: tool for i, tool in enumerate(tool_names)}

b6_result = evaluator.eval(mapping)

print("Solve (b1): ", b1_result.score() / test_dataset.embedding.shape[0])
print("Solve (b2): ", b2_result.score() / test_dataset.embedding.shape[0])
print("Solve (b3): ", b3_result.score() / test_dataset.embedding.shape[0])
print("Solve (b4): ", b4_result.score() / test_dataset.embedding.shape[0])
print("Solve (b5): ", b5_result.score() / test_dataset.embedding.shape[0])
print("Solve (b6): ", b6_result.score() / test_dataset.embedding.shape[0])
print(b6_result)

100%|██████████| 10/10 [00:00<00:00, 341.92it/s]

Solve (b1):  0.93026706231454
Solve (b2):  0.9317507418397626
Solve (b3):  0.9287833827893175
Solve (b4):  0.8887240356083086
Solve (b5):  0.9339762611275965
Solve (b6):  0.9673590504451038
Result( 1304 / 1348 )


# Baseline #7: k-means

In [106]:
from sklearn.cluster import KMeans

K = 4
cluster_assign = KMeans(n_clusters = K).fit_predict(train_embedding)

In [107]:
tools    = []
tool_weights = []

for k in range(K):
    index = cluster_assign == k
    labels = train_dataset.labels[index]
    runtimes = train_dataset.runtimes[index]
    seqp = optimize_portfolio(labels, runtimes)
    seqp = [(train_dataset.label_index[solver], runtime) for solver, runtime in seqp.portfolio]
    tool = ",".join(":".join([str(_b) for _b in b]) for b in seqp)
    print(tool)

    mapping = {instance: seqp for instance in train_dataset.instance_index}
    result  = evaluator.eval(mapping)
    tool_solve = [r[1].startswith(str(r[2]).lower()) for r in result]
    tool_solve = np.array([1 if t else 0 for t in tool_solve])
    tool_runtimes = np.array([r[3] for r in result])
    tool_weight   = tool_runtimes * tool_solve + 9000 * (1 - tool_solve)

    tools.append(tool)
    tool_weights.append(tool_weight)


bmc:900
ki:60,pa:60,symbolic:180,va:240,bmc:300,vaitp:900
ki:120,symbolic:240,vaitp:480,pa:900
bmc:60,pa:60,ki:180,va:300,vaitp:900


In [108]:
for tool in train_dataset.label_index:
    mapping = {instance: tool for instance in train_dataset.instance_index}
    result  = evaluator.eval(mapping)
    tool_solve = [r[1].startswith(str(r[2]).lower()) for r in result]
    tool_solve = np.array([1 if t else 0 for t in tool_solve])
    tool_runtimes = np.array([r[3] for r in result])
    tool_weight   = tool_runtimes * tool_solve + 9000 * (1 - tool_solve)

    tools.append(tool)
    tool_weights.append(tool_weight)

In [109]:
rank_pos    = []
rank_weight = []

for i in range(len(tools) - 1):
    for j in range(i + 1, len(tools)):
        rank_pos.append((i, j))
        rank_weight.append((tool_weights[j] - tool_weights[i]))

rank_weight = np.stack(rank_weight).transpose()
rank_labels, rank_sample_weight = np.clip(np.sign(rank_weight), 0, 1), np.abs(rank_weight)

In [110]:
from tqdm import trange
from sklearn.linear_model import LogisticRegression

models = []
for i in trange(rank_labels.shape[1]):
    labels = rank_labels[:, i] 
    sample_weight = rank_sample_weight[:, i]

    if labels.min() != labels.max():
        clf    =  LogisticRegression(C = 0.1, max_iter = 10_000)
        clf.fit(train_embedding, labels, sample_weight = sample_weight)
    else:
        clf = labels.min()

    models.append(clf)

100%|██████████| 45/45 [00:17<00:00,  2.58it/s]


In [111]:
# Compute new predictions
predictions = []
for i in trange(len(models)):
    model = models[i]
    try:
        prediction = model.predict_proba(test_embedding)[:, 1]
    except Exception:
        prediction = np.array([model] * test_embedding.shape[0])
    predictions.append(prediction)

predictions = np.stack(predictions).transpose()

scores = np.zeros((predictions.shape[0], len(tools)))
for i, (p, k) in enumerate(rank_pos):
    pred = predictions[:, i]

    scores[:, p] += pred
    scores[:, k] += (1 - pred)

selection = scores.argmax(axis = 1)
selection

100%|██████████| 45/45 [00:00<00:00, 602.34it/s]


array([3, 4, 4, ..., 9, 6, 3])

In [112]:
# Evaluate
tool_names   = [tools[i] for i in selection]
mapping = {test_dataset.instance_index[i]: tool for i, tool in enumerate(tool_names)}

b7_result = evaluator.eval(mapping)

print("Solve (b1): ", b1_result.score() / test_dataset.embedding.shape[0])
print("Solve (b2): ", b2_result.score() / test_dataset.embedding.shape[0])
print("Solve (b3): ", b3_result.score() / test_dataset.embedding.shape[0])
print("Solve (b4): ", b4_result.score() / test_dataset.embedding.shape[0])
print("Solve (b5): ", b5_result.score() / test_dataset.embedding.shape[0])
print("Solve (b6): ", b6_result.score() / test_dataset.embedding.shape[0])
print("Solve (b7): ", b7_result.score() / test_dataset.embedding.shape[0])
print(b7_result)

Solve (b1):  0.93026706231454
Solve (b2):  0.9317507418397626
Solve (b3):  0.9287833827893175
Solve (b4):  0.8887240356083086
Solve (b5):  0.9339762611275965
Solve (b6):  0.9673590504451038
Solve (b7):  0.9658753709198813
Result( 1302 / 1348 )
